# Toy version

In [1]:
import gym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from gym import spaces
from gym_trading_env.environments import TradingEnv

from scripts import *

from tqdm import trange, tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df = pd.read_csv('data/INTC_1Min_2023-08-01_2024-01-31.csv')
df.set_index('timestamp', inplace=True)

df = df.iloc[:len(df) // 10]

In [ ]:
# def train_iRDPG(df, window_size=60, hidden_dim=64,
#                 train_episodes=500, batch_size=8, gamma=0.99,
#                 tau=0.01, expert_demos=None):
    
#     env = POMDPTEnv(df, window_size=window_size)
#     obs_dim = env.observation_space.shape[0]
#     act_dim = env.action_space.n
    
#     agent = iRDPGAgent(obs_dim, act_dim, hidden_dim).to(device)
#     target_agent = iRDPGAgent(obs_dim, act_dim, hidden_dim).to(device)
#     target_agent.load_state_dict(agent.state_dict())
    
#     # Create a replay buffer
#     buffer = RBuffer(max_episodes=1000)
#     print(f"Initialized the environment: window_size={window_size}, \n \
#           the agent: observation dimension: {obs_dim}, action dimension: {act_dim}, \n \
#           and replay buffer.")
    
#     # Collect some demonstration episodes
#     for ep in expert_demos:
#         buffer.add_episode(ep)
#     print(f"Collected {len(expert_demos)} demonstration episodes.")
    
#     # Create "prophetic" array for entire df:
#     prophecy = intraday_greedy_actions(df, window_size=window_size, device=device)
#     print(f"Created the 'prophetic' array for the entire dataset.")
    
#     # Attach them to each Episode so we can do Q-filter BC later
#     for ep in tqdm(buffer.episodes, desc="Attaching Prophetic Actions"):
#         ep.expert_actions = []
#         # naive mapping: the environment index starts at window_size for step=0
#         # so step i -> global index = window_size + i
#         idx_global = window_size
#         for t in range(len(ep.obs)):
#             if idx_global < len(prophecy):
#                 ep.expert_actions.append(prophecy[idx_global])
#             else:
#                 ep.expert_actions.append(0)
#             idx_global += 1
#     print(f"Attached the 'prophetic' actions to demonstration episodes.")
    
#     optimizer = torch.optim.Adam(agent.parameters(), lr=1e-3)
#     ce = nn.CrossEntropyLoss()
    
#     def soft_update(net, net_targ, tau):
#         for p, p_targ in zip(net.parameters(), net_targ.parameters()):
#             p_targ.data.copy_(tau * p.data + (1.0 - tau) * p_targ.data)
    
#     def update_agent(batch_eps):
#         """
#         Unroll entire episodes on GPU for BPTT.
#         """
#         total_loss = 0.0
#         agent.train()
#         optimizer.zero_grad()
        
#         for ep in tqdm(batch_eps, desc="Updating agent"):
#             obs_seq = torch.tensor(ep.obs, dtype=torch.float, device=device)
#             actions_seq = torch.tensor(ep.actions, dtype=torch.long, device=device)
#             rewards_seq = torch.tensor(ep.rewards, dtype=torch.float, device=device)
#             exp_seq = torch.tensor(ep.expert_actions, dtype=torch.long, device=device)
            
#             # shape => [1, T, obs_dim]
#             obs_seq = obs_seq.unsqueeze(0) 
            
#             logits, Q_vals, _, _ = agent(obs_seq, actions_seq)  # [1,T,act_dim]
#             logits = logits[0]  # [T,act_dim]
#             Q_vals = Q_vals[0] # [T,act_dim]
#             T_len = len(actions_seq)
            
#             # Critic loss: basic 1-step TD
#             td_errors = []
#             for t in range(T_len):
#                 q_sa = Q_vals[t, actions_seq[t]]
#                 if t < T_len-1 and not ep.done_flags[t]:
#                     # next action from agent policy
#                     next_a = logits[t+1].argmax().item()
#                     q_next = Q_vals[t+1, next_a].detach()
#                     target = rewards_seq[t] + gamma * q_next
#                 else:
#                     target = rewards_seq[t]
#                 td_errors.append(target - q_sa)
            
#             td_errors = torch.stack(td_errors)
#             critic_loss = (td_errors**2).mean()
            
#             # Actor loss: - Q(s, a_pi)
#             pi = torch.softmax(logits, dim=-1)
#             Q_expected = (pi * Q_vals).sum(dim=-1)
#             actor_loss = - Q_expected.mean()
            
#             # Behavior cloning with Q-filter
#             a_agent = logits.argmax(dim=-1)
#             Q_expert = Q_vals[torch.arange(T_len), exp_seq]
#             Q_agent  = Q_vals[torch.arange(T_len), a_agent]
#             mask = (Q_expert > Q_agent).float()
            
#             bc_loss_t = ce(logits, exp_seq)  # cross-entropy stepwise
#             bc_loss   = (bc_loss_t * mask).mean()
            
#             ep_loss = critic_loss + actor_loss + bc_loss
#             ep_loss.backward(retain_graph=True)
#             total_loss += ep_loss.item()
        
#         nn.utils.clip_grad_norm_(agent.parameters(), 5.0)
#         optimizer.step()
        
#         soft_update(agent, target_agent, tau)
        
#         return total_loss / len(batch_eps)
    
#     # main training loop
#     update_trange = trange(train_episodes, desc="Training")
#     for ep_i in update_trange:
#         new_ep = collect_episode(env, agent=agent, noise=True)
#         # attach prophecy to new episode
#         new_ep.expert_actions = []
#         idx_global = window_size
#         for t in range(len(new_ep.obs)):
#             if idx_global < len(prophecy):
#                 new_ep.expert_actions.append(prophecy[idx_global])
#             else:
#                 new_ep.expert_actions.append(0)
#             idx_global += 1
        
#         buffer.add_episode(new_ep)
        
#         if len(buffer) >= batch_size:
#             batch_eps = buffer.sample(batch_size)
#             loss_val = update_agent(batch_eps)
#         else:
#             loss_val = 0.0
#         update_trange.set_postfix(loss=loss_val)
        
#         print(f"Episode {ep_i+1}/{train_episodes}, Loss={loss_val:.4f}")
    
#     return agent, target_agent

In [ ]:
# def train_iRDPG(
#     df,
#     n_train_episodes=100,
#     n_demo_episodes=20,
#     window_size=60,
#     batch_size=4,
#     train_iterations_per_episode=10,
#     gamma=0.99,
#     lr_actor=1e-4,
#     lr_critic=1e-3,
#     expert_demos=None,
#     device="cuda"
# ):
#     """
#     Example training loop for iRDPG.

#     :param df:          (pandas.DataFrame) containing columns ['open','high','low','close'] for the environment
#     :param n_train_episodes: (int) total training episodes
#     :param n_demo_episodes:  (int) how many demonstration episodes to collect from Dual Thrust
#     :param window_size: (int) lookback window for environment
#     :param demo_k1:     (float) parameter K1 for Dual Thrust
#     :param demo_k2:     (float) parameter K2 for Dual Thrust
#     :param batch_size:  (int) batch size for sampling episodes from replay
#     :param train_iterations_per_episode: (int) how many gradient updates to do after each collection
#     :param gamma:       (float) discount factor
#     :param lr_actor:    (float) learning rate for actor
#     :param lr_critic:   (float) learning rate for critic
#     :param device:      (str) "cuda" or "cpu"
#     """

#     # 1) Create the environment
#     env = POMDPTEnv(df, window_size=window_size)

#     # 2) Create the agent (iRDPGAgent is your GRU-based Actor-Critic)
#     obs_dim = env.observation_space.shape[0]   # 4 + 4*window_size
#     act_dim = env.action_space.n               # 3 (0: hold, 1: long, 2: short)
#     agent = iRDPGAgent(obs_dim=obs_dim, action_dim=act_dim, hidden_dim=64, device=device)
#     agent.to(device)

#     replay_buffer = RBuffer(max_episodes=1000, device=device)
#     # 3) Create the replay buffer
#     for ep in expert_demos:
#         replay_buffer.add_episode(ep)

#     # 4) [Optional] Collect demonstration data from Dual Thrust.
#     #    In your code, you had a function `collect_demonstrations`.
#     #    We'll show how to use it. If you prefer a different demon policy,
#     #    just adapt accordingly.

#     # 5) Create separate optimizers for actor and critic
#     actor_params = list(agent.actor_gru.parameters()) + list(agent.actor_fc.parameters())
#     critic_params = list(agent.critic_gru.parameters()) + list(agent.critic_fc.parameters())

#     actor_optimizer = torch.optim.Adam(actor_params, lr=lr_actor)
#     critic_optimizer = torch.optim.Adam(critic_params, lr=lr_critic)

#     # 6) Utility function for computing the RDPG losses
#     #    We do "backprop through time" on entire episodes or truncated sub-sequences
#     #    For simplicity, we'll do full-episode BPTT on small batch_size.
#     def compute_losses(batch_eps):
#         """
#         Compute actor and critic losses for a list of episodes (batch_eps).
#         Each item is an Episode object with obs, actions, rewards, next_obs, done_flags.
#         """
#         # We'll store MSE for critic, policy gradient for actor.
#         # Because we have a recurrent architecture, we need to pass each
#         # time step in sequence into the GRU.
#         total_critic_loss = 0.0
#         total_actor_loss = 0.0

#         for ep in batch_eps:
#             # (T, obs_dim)
#             obs_seq = ep.obs
#             actions_seq = ep.actions
#             rewards_seq = ep.rewards
#             done_seq = ep.done_flags

#             T_steps = len(obs_seq)
#             # We'll accumulate hidden states as we unroll the sequence
#             h_actor = torch.zeros(1, 1, 64, device=device)   # (num_layers, batch=1, hidden_dim)
#             h_critic = torch.zeros(1, 1, 64, device=device)

#             # We collect all predicted Q-values, all chosen actions, etc.
#             critic_loss = 0.0
#             actor_loss = 0.0

#             # We'll do discount factor manually or incorporate into reward
#             # typical RDPG: we do a multi-step target with gamma
#             # We'll keep track of the running return for the target
#             # For simplicity, let's do a 1-step TD target (though multi-step is possible).
#             for t in range(T_steps):
#                 # Turn single obs into shape (1,1,obs_dim)
#                 obs_t = obs_seq[t].view(1,1,-1)
#                 # Pass into agent
#                 with torch.no_grad():
#                     # no_grad for deciding the action that was actually taken
#                     dummy_action = torch.zeros((1,1), dtype=torch.long, device=device)
#                     q_logits, q_value_all, h_actor_next, h_critic_next = agent.forward(
#                         obs_t, dummy_action, h_actor, h_critic
#                     )
#                 # the action was discrete 0,1,2. Let's get the Q-value for that action:
#                 act_idx = actions_seq[t]
#                 # Q for the last time-step in the sequence dimension
#                 # q_value_all is shape (1, action_dim) if we took the last index
#                 Q_chosen = q_value_all[0, act_idx]

#                 # to do 1-step TD, we need r + gamma * max Q(next)
#                 # but iRDPG is typically deterministic, so let's do r + gamma * Q(next, a_next)
#                 reward_t = rewards_seq[t]
#                 done_t = done_seq[t]

#                 # Next obs (if not done)
#                 if t < (T_steps - 1) and not done_t:
#                     next_obs_t = ep.obs[t+1].view(1,1,-1)
#                     with torch.no_grad():
#                         # get next action from actor
#                         next_logits, _, h_actor_next2, h_critic_next2 = agent.forward(
#                             next_obs_t, dummy_action, h_actor_next, h_critic_next
#                         )
#                         next_probs = torch.softmax(next_logits[0,0,:], dim=-1)
#                         a_next_idx = next_probs.argmax().item()
#                         # evaluate Q(next_obs, a_next)
#                         # we need Q from the "target" network or from the same network with no_grad
#                         # for simplicity, we'll reuse same agent as a target
#                         _, q_value_next_all, _, _ = agent.forward(
#                             next_obs_t, dummy_action, h_actor_next, h_critic_next
#                         )
#                         Q_next = q_value_next_all[0, a_next_idx]
#                     td_target = reward_t + gamma * Q_next
#                 else:
#                     td_target = reward_t

#                 # Critic loss: MSE
#                 critic_loss_t = (td_target - Q_chosen)**2
#                 critic_loss += critic_loss_t

#                 # For actor loss, we want to move the actor's chosen action to maximize Q
#                 # We'll do a second forward pass that requires grad for the actor
#                 obs_t.requires_grad = True
#                 q_logits_live, q_value_all_live, h_actor_live, h_critic_live = agent.forward(
#                     obs_t, dummy_action, h_actor, h_critic
#                 )
#                 # The actor's chosen action is argmax of logits
#                 a_probs = torch.softmax(q_logits_live[0,0,:], dim=-1)
#                 # We do a “differentiable” version: the Q for the distribution of actions under the policy
#                 # but iRDPG is typically picking a discrete action via argmax in inference.
#                 # A simpler approach is to do a policy gradient that picks the best action w.r.t Q:
#                 Q_actor = torch.sum(a_probs * q_value_all_live[0], dim=-1)
#                 # We want to maximize Q_actor => minimize negative Q_actor
#                 actor_loss_t = -Q_actor
#                 actor_loss += actor_loss_t

#                 # Move forward hidden states
#                 h_actor = h_actor_next
#                 h_critic = h_critic_next

#             # Sum or average across timesteps
#             critic_loss = critic_loss / T_steps
#             actor_loss = actor_loss / T_steps

#             total_critic_loss += critic_loss
#             total_actor_loss += actor_loss

#         # Average over episodes in the batch
#         total_critic_loss /= len(batch_eps)
#         total_actor_loss /= len(batch_eps)

#         return total_critic_loss, total_actor_loss

#     # 7) Main training loop
#     print("Starting iRDPG training ...")
#     for ep_idx in range(n_train_episodes):
#         # Collect 1 on-policy episode from the environment
#         ep_data = Episode()
#         obs = env.reset()
#         done = False

#         # You can add Gaussian or epsilon-greedy noise if needed
#         while not done:
#             # Decide action from agent
#             # agent.act() expects (obs, h_actor) but for simplicity we track hidden states externally
#             # or we can keep it in the agent. We'll do a simple approach:
#             action, _ = agent.act(obs, torch.zeros(1,1,64,device=device))

#             next_obs, reward, done, _info = env.step(action)

#             ep_data.obs.append(obs)
#             ep_data.actions.append(action)
#             ep_data.rewards.append(reward)
#             ep_data.done_flags.append(done)

#             obs = next_obs

#         # Store the newly collected episode
#         replay_buffer.add_episode(ep_data)

#         # Do some training updates
#         for _ in range(train_iterations_per_episode):
#             batch_eps = replay_buffer.sample(batch_size)
#             critic_loss, actor_loss = compute_losses(batch_eps)

#             # Backprop critic
#             critic_optimizer.zero_grad()
#             critic_loss.backward()
#             critic_optimizer.step()

#             # Backprop actor
#             actor_optimizer.zero_grad()
#             actor_loss.backward()
#             actor_optimizer.step()

#         if (ep_idx+1) % 10 == 0:
#             print(f"[Episode {ep_idx+1}/{n_train_episodes}] Critic Loss: {critic_loss.item():.4f}, "
#                   f"Actor Loss: {actor_loss.item():.4f}")

#     print("Training complete!")
#     return agent



In [ ]:
expert_demos = collect_demonstrations(df, window_size=60, n_episodes=1)
# torch.save(expert_demos, 'expert_demos.pt')
# print("Expert demonstrations collected and saved.")

In [ ]:
t = 1_000

# trained, target = train_iRDPG(df=df, window_size=60, train_episodes=t, batch_size=8, gamma=0.99, tau=0.01, expert_demos=expert_demos)

trained_agent = train_iRDPG(df, n_train_episodes=t, n_demo_episodes=1, window_size=60, batch_size=8, train_iterations_per_episode=10, gamma=0.99, lr_actor=1e-4, lr_critic=1e-3, expert_demos=expert_demos, device="cuda")